#  How to Run GEODYN using PYGEODYN

## Overall Steps
   1. Construct a YAML file that contains the `Run_Settings`
   2. Call an instance of the `Pygeodyn` Object that is loaded with the run settings
   3. Use the `Pygeodyn.RUN_GEODYN()` method to run GEODYN with the requested settings

In [10]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Construct a YAML file that contains the `Run_Settings`


In [4]:
'''
---
###  Run Settings Test File 

# Need to figure out how to use this file to overwrite the Setting hardcoded in ICESat2class if they are here.

### the epoch end time automatically controls the prediction end time
          ### FORMAT: 'YYMMDD HHMMSS.SSSSSSS'
epoch_start         : [
                        '181109 000000.0000000',
                        '181110 000000.0000000',
                        ]

epoch_end           : [ 
                        '181110 000000.0000000',
                        '181111 000000.0000000',
                       ]

### SPECIFY ARC NAMING CONVENTION
arc_length     : '24hr'
file_string    : 'EXAMPLE_TEST_NoCD_Arc24'

cd_adjustment_boolean  : False
total_hours_in_run     : None
hours_between_cd_adj   : None


### RUN PARAMETERS
run_params:
    {arc             : [
                        '2018.313',
                        '2018.314',
                        ],
                        
    satellite        : 'icesat2' ,
    den_model        : 'msis2'  ,
    SpecialRun_name  : '_valid_ctrlrun',  
    verbose          :  False,
    accels           :  False,
    geodyn_StepSize  : 10.,
    }
   
model_data_path : None
request_data:  [ #'AdjustedParams',
                'Trajectory_orbfil',
                'Density',
#                 'Residuals_obs',
                'Residuals_summary',
                'RunSummary',
                                  ]


### SETUP FILE OPTIONS 
cards_to_remove : ['ACCEL9',
                   'ORBTVU',
                   'RESID',
                   'CON9PA',
                  ]



#### ----------------------------------------------------------------------------
#### Options that are hardcoded defaults by ICESAT2 class
#### ----------------------------------------------------------------------------
#### The name of the satellite
SATELLITE_dir  : 'icesat2'
#### Satellites COSPAR ID number
SATID          : '1807001'
#### Datatype being used as tracking data
DATA_TYPE      : 'PCE'
#### Naming convention requested for different gravity types
grav_id        : '' 
#### Do you want to use the updated DRHODZ calculation? (recommended)
options_in     : {'DRHODZ_update':True}  
#### ----------------------------------------------------------------------------
####### File Choices:
#### G2B file name
g2b_file       : 'g2b_pce_fullset_nomaneuver'  
#### Atmospheric Gravity file name
atgrav_file    : 'ATGRAV.glo-3HR_20160101-PRESENT_9999_AOD1B_0006.0090'
#### Planetary Ephemeris file name
ephem_file     : 'ephem1430.data_2025'
#### Gravity field file name
gravfield_file : 'eigen-6c.gfc_20080101_do_200_fix.grv'
#### PCE Ascii textfile name
StateVector_epochs_datafile  : '/data/data_geodyn/inputs/icesat2/setups/PCE_ascii.txt'
'''
print()


In [2]:
run_settings = '/data/geodyn_proj/GETTING_STARTED/example_run_settings.yaml'

2. Call an instance of the `Pygeodyn` Object that is loaded with the run settings


In [ ]:
### Load the data into an object
RunObject = Pygeodyn(run_settings)

3. Use the `Pygeodyn.RUN_GEODYN()` method to run GEODYN with the requested settings

In [ ]:
RunObject.RUN_GEODYN()

## What is happening behind the scenes?

1. A Pygeodyn object is initialized with the specified run settings saved to a YAML
    - `/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN.py`
        1. The `RunObject` is loaded such that it inherits a Satellite Class(either ICESat2 or Starlette) settings.  This means that the methods (functions attributed to an class) that have been written into the ICESat2 Class will overwrite the functions in the base classes.  The base classes are `PygeodynController` and `PygeodynReader` and they are each inherited into the satellite classes.
        2. So after the `Object = Pygeodyn(run_settings)` call, the following happens:
            - The Pygeodyn class inherits the `Inherit_Icesat2` class which tells Pygeodyn to adopt the functions in the `Satellite_ICESat2` class (`/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_ICESat2.py`).
            - The `Satellite_ICESat2` class inherits both `PygeodynController` and `PygeodynReader`.  With this step, the object initialized by calling `Object = Pygeodyn(run_settings)` now contains all of the functions in `PygeodynController` and `PygeodynReader` and additionally, any functions that are written into `Satellite_ICESat2` with identical names overwrite those in the Controller and Reader.
            
 - Note--- each time a class is called (in the case of `Pygeodyn`) or inherited (the case for `Satellite_ICESat2` and then `PygeodynController` and `PygeodynReader`) the `def __init__()` function runs.  It is in this location that we put anything we want to have happen for each instance of the Class (like any class specific global variables).  If you don't want anything to happen, you can put a pass.

    ```python
    class example_class():
        def __init__(self):  
            pass
    ```




2. Call the `RUN_GEODYN()` method which is now attributed to the object (`RunObject`) that is initialized with the settings and classes as defined above.

    - Explicitly stated again: `RunObject` is an object of the `Pygeodyn` class that inherited the class `Satellite_ICESat2`, which itself inherited `PygeodynController` and `PygeodynReader`).  Therefore `RunObject` contains all of the methods (function for a class) contained in all of those classes.
    -  `RUN_GEODYN()` is called thru these classes and its method is located at the bottom of the PYGEODYN_Control file: `/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_Control.py` 




